In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Titanic_survival.csv")

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived']

In [5]:
df["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [6]:
X.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
cat = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("encoder",OneHotEncoder(handle_unknown='ignore'))])
num = Pipeline(steps=[("imputer",SimpleImputer(strategy="mean"))])
preprocessor  = ColumnTransformer(transformers=[
    ('num', num, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
    ('cat', cat, ['Embarked','Sex'])
])

In [24]:
from sklearn.linear_model import LogisticRegression
pipe = Pipeline([
    ("prepocessor",preprocessor),
    ("model",LogisticRegression())
])

In [25]:
from sklearn.model_selection import train_test_split,GridSearchCV
parameters = {'model__penalty':["l1","l2","elasticnet"],
              'model__C':[0.2,1,3,6,10,15,30,50],
              'model__solver': ['liblinear', 'saga'],
              "model__max_iter": [100,500,1000],
              "model__l1_ratio": [0.1, 0.5, 0.9]}
grid = GridSearchCV(pipe,parameters,scoring="roc_auc",cv=5)

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [27]:
grid.fit(x_train,y_train)

c:\jupyter\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\jupyter\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\jupyter\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\jupyter\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\jupyter\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\jupyter\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1221: UserW

,estimator,Pipeline(step...egression())])
,param_grid,"{'model__C': [0.2, 1, ...], 'model__l1_ratio': [0.1, 0.5, ...], 'model__max_iter': [100, 500, ...], 'model__penalty': ['l1', 'l2', ...], ...}"
,scoring,'roc_auc'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [28]:
grid.best_estimator_

,steps,"[('prepocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [29]:
grid.best_params_

{'model__C': 1,
 'model__l1_ratio': 0.1,
 'model__max_iter': 100,
 'model__penalty': 'l1',
 'model__solver': 'liblinear'}

In [30]:
grid.best_score_

np.float64(0.8446009811920347)

In [31]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
y_pred = grid.predict(x_test)
print(f"Accuracy : {accuracy_score(y_test,y_pred)}")
print(f"Classification Report : {classification_report(y_test,y_pred)}")
print(f"confusion matrix : {confusion_matrix(y_test,y_pred)}")

Accuracy : 0.7932960893854749
Classification Report :               precision    recall  f1-score   support

           0       0.81      0.85      0.83       105
           1       0.77      0.72      0.74        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179

confusion matrix : [[89 16]
 [21 53]]
